# Custom Components

This is a guide on how to build a simple app and custom component spec and
launch it via two different schedulers.

See the [Quickstart Guide](quickstart.md) for installation and basic usage.

## Builtins

Before writing a custom component, check if any of the builtin components
satisfy your needs. TorchX provides a number of builtin components with premade
images. You can discover them via:

In [1]:
%%sh
torchx builtins

Found 11 builtin components:
  1. dist.ddp
  2. dist.spmd
  3. metrics.tensorboard
  4. serve.torchs

erve
  5. utils.binary
  6. utils.booth
  7. utils.copy
  8. utils.echo
  9. utils.python
 10. utils

.sh
 11. utils.touch


You can use these either from the CLI, from a pipeline or programmatically like
you would any other component.

In [2]:
%%sh
torchx run utils.echo --msg "Hello :)"

torchx 2025-11-13 21:32:57 INFO     Tracker configurations: {}


torchx 2025-11-13 21:32:57 INFO     Workspace `/home/runner/work/torchx/torchx/docs/source` resolved

 to filesystem path `/home/runner/work/torchx/torchx/docs/source`


torchx 2025-11-13 21:35:21 INFO     Building workspace docker image (this may take a while)...


torchx 2025-11-13 21:35:21 INFO     Step 1/4 : ARG IMAGE


...

torchx 2025-11-13 21:35:34 INFO     role[0]=echo updated with new image to include workspace changes

torchx 2025-11-13 21:35:34 INFO     Waiting for the app to finish...


echo/0 Hello :)


torchx 2025-11-13 21:35:35 INFO     Job finished: SUCCEEDED


local_docker://torchx/echo-z6fgnff10pvpq


## Hello World

Lets start off with writing a simple "Hello World" python app. This is just a
normal python program and can contain anything you'd like.

<div class="admonition note">
<div class="admonition-title">Note</div>
This example uses Jupyter Notebook `%%writefile` to create local files for
example purposes. Under normal usage you would have these as standalone files.
</div>

In [3]:
%%writefile my_app.py

import sys
import argparse

def main(user: str) -> None:
    print(f"Hello, {user}!")

if __name__ == "__main__":
    parser = argparse.ArgumentParser(
        description="Hello world app"
    )
    parser.add_argument(
        "--user",
        type=str,
        help="the person to greet",
        required=True,
    )
    args = parser.parse_args(sys.argv[1:])

    main(args.user)

Writing my_app.py


Now that we have an app we can write the component file for it. This function
allows us to reuse and share our app in a user friendly way.

We can use this component from the `torchx` cli or programmatically as part of a
pipeline.

In [4]:
%%writefile my_component.py

import torchx.specs as specs

def greet(user: str, image: str = "my_app:latest") -> specs.AppDef:
    return specs.AppDef(
        name="hello_world",
        roles=[
            specs.Role(
                name="greeter",
                image=image,
                entrypoint="python",
                args=[
                    "-m", "my_app",
                    "--user", user,
                ],
            )
        ],
    )

Writing my_component.py


We can execute our component via `torchx run`. The `local_cwd` scheduler
executes the component relative to the current directory.

In [5]:
%%sh
torchx run --scheduler local_cwd my_component.py:greet --user "your name"

torchx 2025-11-13 21:35:35 INFO     Tracker configurations: {}


torchx 2025-11-13 21:35:35 INFO     Log directory not set in scheduler cfg. Creating a temporary log

 dir that will be deleted on exit. To preserve log directory set the `log_dir` cfg option
torchx 202

5-11-13 21:35:35 INFO     Log directory is: /tmp/torchx_y4ljm2xq


torchx 2025-11-13 21:35:35 INFO     Waiting for the app to finish...


greeter/0 Hello, your name!


torchx 2025-11-13 21:35:36 INFO     Job finished: SUCCEEDED


local_cwd://torchx/hello_world-cpzqff9bdwqwt


If we want to run in other environments, we can build a Docker container so we
can run our component in Docker enabled environments such as Kubernetes or via
the local Docker scheduler.

<div class="admonition note">
<div class="admonition-title">Note</div>
This requires Docker installed and won't work in environments such as Google
Colab. If you have not done so already follow the install instructions on:
[https://docs.docker.com/get-docker/](https://docs.docker.com/get-docker/)</a>
</div>

In [6]:
%%writefile Dockerfile.custom

FROM ghcr.io/pytorch/torchx:0.1.0rc1

ADD my_app.py .

Writing Dockerfile.custom


Once we have the Dockerfile created we can create our docker image.

In [7]:
%%sh
docker build -t my_app:latest -f Dockerfile.custom .

#0 building with "default" instance using docker driver

#1 [internal] load build definition from Do

ckerfile.custom
#1 transferring dockerfile: 99B done
#1 DONE 0.0s

#2 [internal] load metadata for g

hcr.io/pytorch/torchx:0.1.0rc1


#2 DONE 0.5s



#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [internal] load 

...

xtracting sha256:88d87059c913e67971846680d4032b75f96f599f8a67062f668fed6471fc2968 0.0s done


#5 DONE 80.3s

#6 [2/2] ADD my_app.py .
#6 DONE 0.0s

#7 exporting to image
#7 exporting layers


#7 exporting layers 4.3s done


#7 writing image sha256:2a2687bdcd1ab09d8186b9c6c3db47636297d60d692bf7469419cce0889f6624 done
#7 nam

ing to docker.io/library/my_app:latest done
#7 DONE 4.3s


We can then launch it on the local scheduler.

In [8]:
%%sh
torchx run --scheduler local_docker my_component.py:greet --image "my_app:latest" --user "your name"

torchx 2025-11-13 21:37:07 INFO     Tracker configurations: {}


torchx 2025-11-13 21:37:07 INFO     Workspace `/home/runner/work/torchx/torchx/docs/source` resolved

 to filesystem path `/home/runner/work/torchx/torchx/docs/source`


torchx 2025-11-13 21:37:07 WARNING  failed to pull image my_app:latest, falling back to local: 404 C

lient Error for http+docker://localhost/v1.48/images/create?tag=latest&fromImage=my_app: Not Found (

...

ges


torchx 2025-11-13 21:37:16 INFO     Waiting for the app to finish...


greeter/0 Hello, your name!


torchx 2025-11-13 21:37:17 INFO     Job finished: SUCCEEDED


local_docker://torchx/hello_world-jspqfzqc5r1pkd


If you have a Kubernetes cluster you can use the
[Kubernetes scheduler](schedulers/kubernetes.rst) to launch this on the cluster
instead.


```sh
$ docker push my_app:latest
$ torchx run --scheduler kubernetes my_component.py:greet --image "my_app:latest" --user "your name"
```
